In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt


### 전국 산림 5차 임상도 원본 데이터 읽기 

In [3]:
folder_ids = [11, 26, 27, 28, 29, 30, 31, 36, 41, 43, 44, 46, 47, 48, 50, 51, 52]
loaded = []
for id in folder_ids:
    gdf = gpd.read_file(f"./전국 산림 5차 임상도 원본 데이터/{id}/{id}.shp")
    loaded.append(gdf)

loaded[2].drop(columns = ["UserID"], inplace =True)
loaded[12].drop(columns = ["UserID"], inplace =True)



### 원본 데이터들 GeoDataFrame으로 합치기 

In [4]:
merged = pd.concat(loaded, ignore_index=True)
merged = gpd.GeoDataFrame(merged, crs=loaded[0].crs)

### 용량 축소를 위해 geometry 값 simplify

In [5]:
merged["geo_simple"] = merged["geometry"].simplify(tolerance = 100)
merged.set_geometry('geo_simple', inplace=True)
merged.drop(columns = ["Shape_Leng", "Shape_Area", "SMBL_CD", "geometry"], inplace=True)


In [7]:
merged.head()

,FIFTH_FRTP,DMCLS_CD,AGCLS_CD,DNST_CD,geo_simple
0,H,2,4,B,"POLYGON ((948832.506 1938334.243, 948821.829 1..."
1,H,2,4,B,"POLYGON ((948400.28 1943550.352, 948436.09 194..."
2,H,2,4,B,"POLYGON ((948917.086 1943877.663, 949068.461 1..."
3,H,2,5,B,"POLYGON ((947921.377 1939678.505, 948085.529 1..."
4,H,2,5,B,"POLYGON ((948903.335 1939687.335, 949128.625 1..."


In [9]:
merged.tail()

,FIFTH_FRTP,DMCLS_CD,AGCLS_CD,DNST_CD,geo_simple
501718,Q,2,4,C,"POLYGON ((1033189.479 1777946.008, 1033491.85 ..."
501719,Q,2,5,B,"POLYGON ((1032881.538 1777306.48, 1033089.581 ..."
501720,Q,2,5,C,"POLYGON ((1033589.258 1776815.165, 1033476.936..."
501721,Q,2,5,C,"POLYGON ((1033799.15 1777874.173, 1033799.4 17..."
501722,Q,2,5,C,"POLYGON ((1033798.322 1778090.079, 1033798.691..."


###결측치 제거

In [10]:
merged.dropna(subset = ["FIFTH_FRTP", "DMCLS_CD", "AGCLS_CD", "DNST_CD"], inplace= True)
merged.reset_index(inplace = True)

merged.tail()

,index,FIFTH_FRTP,DMCLS_CD,AGCLS_CD,DNST_CD,geo_simple
445510,501718,Q,2,4,C,"POLYGON ((1033189.479 1777946.008, 1033491.85 ..."
445511,501719,Q,2,5,B,"POLYGON ((1032881.538 1777306.48, 1033089.581 ..."
445512,501720,Q,2,5,C,"POLYGON ((1033589.258 1776815.165, 1033476.936..."
445513,501721,Q,2,5,C,"POLYGON ((1033799.15 1777874.173, 1033799.4 17..."
445514,501722,Q,2,5,C,"POLYGON ((1033798.322 1778090.079, 1033798.691..."


### 사용할 수 있는 gpkg로 저장

In [11]:
merged.to_file("merged100_unconverted.gpkg", driver="GPKG")

### 식생 종류 코드 변환

# 각 코드를 이름으로 print함수 

# 인공림도 포함한 변환 함수와, 인공림은 제외한 변환 함수(2)

In [6]:
forest_type_dict = {
    # 침엽수
    'D': '소나무림 (자연산, 소나무 ≥ 75%)',
    'PD': '소나무 인공림 (소나무 ≥ 75%)',
    'PK': '잣나무림 (잣나무 ≥ 75%)',
    'PL': '낙엽송림 (낙엽송 ≥ 75%)',
    'PR': '리기다소나무림 (리기다소나무 ≥ 75%)',
    'PT': '해송림 (해송 ≥ 75%)',
    'PC': '침엽수인공림 (침엽수 ≥ 75%)',
    'PCr': '삼나무림 (삼나무 ≥ 75%)',
    'PCo': '편백나무림 (편백나무 ≥ 75%)',

    # 활엽수
    'Q': '참나무림 (자연산, 참나무 ≥ 75%)',
    'H': '활엽수림 (자연산, 활엽수 ≥ 75%)',
    'PH': '활엽수인공림 (활엽수 ≥ 75%)',
    'B': '죽림 (대나무 ≥ 75%)',
    'PCa': '밤나무림 (밤나무 ≥ 75%)',

    # 혼효림
    'M': '침·활 혼효림 (자연산, 침엽·활엽 각각 25% 이상 75% 미만)',


    # 무임목지 / 기타
    'O': '미립목지 (지피 및 수관 피복도 ≥ 50%)',
    'E': '황폐지 (피복도 ≤ 50%, 침계 발달 시 ≤ 60%)',
    'L': '경작지 (과수원, 농지 등)',
    'R': '제지 (도로, 암석지, 묘지, 군사시설 부지)',
    'W': '하천/소택지/수면',
    'LP': '초지 (목장)',
    'F': '산불피해지 (산불로 훼손된 산림)'
}

# 침엽수림, 활엽수림, 혼효림과 그 외로 나눔.
coniferous = ['D', 'PD', 'PK', 'PL', 'PR', 'PT', 'PC', 'PCr', 'PCo']
broadleaf = ['Q', 'H', 'PH', 'B', 'PCa']
mixed = ['M']
other = ['O', 'E', 'L', 'R', 'W', 'LP', 'F']

# 각 코드 이름으로 변환 함수.
def print_frtp(x):
    if  x in forest_type_dict:
        print(forest_type_dict[x])
    else:
        print("Unidentified type.")

# 자연림, 인공림 둘다 변환
def convert_frtp(x):
    if x in coniferous:
        #침엽수림 
        return "침엽수림"
    elif x in broadleaf:
        #활엽수림 
        return "활엽수림"
    elif x in mixed:
        # 혼효림
        return '혼효림'
    elif x in other:
        # 식생 없음 
        return None
    else:
        # Nan
        return None
    
# 자연림만 변환
def convert_frtp2(x):
    if x == "D":
        #침엽수림 
        return "침엽수림"
    elif ((x == "H")|(x == "Q")):
        #활엽수림 
        return "활엽수림"
    elif x == "M":
        # 혼효림
        return "혼효림"
    else:
        # Nan
        return None

### 추출, 삼림 종류 별로 변환.

In [7]:
frst_gdf = gpd.read_file("./modified_geopackage/merged100_unconverted.gpkg")

In [8]:
frst_gdf["frtp_CD"] = frst_gdf["FIFTH_FRTP"].apply(convert_frtp)
frst_gdf = frst_gdf[frst_gdf["frtp_CD"].notna()]
frst_gdf.head()

,index,FIFTH_FRTP,DMCLS_CD,AGCLS_CD,DNST_CD,geometry,frtp_CD
0,0,H,2,4,B,"MULTIPOLYGON (((948832.506 1938334.243, 948821...",활엽수림
1,1,H,2,4,B,"MULTIPOLYGON (((948400.28 1943550.352, 948436....",활엽수림
2,2,H,2,4,B,"MULTIPOLYGON (((948917.086 1943877.663, 949068...",활엽수림
3,3,H,2,5,B,"MULTIPOLYGON (((947921.377 1939678.505, 948085...",활엽수림
4,4,H,2,5,B,"MULTIPOLYGON (((948903.335 1939687.335, 949128...",활엽수림


### 삼림 종류 단순화한 데이터 따로 저장.

In [9]:
frst_gdf.to_file("merged100_converted(artif_included).gpkg", driver="GPKG")